# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [18]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

resource_url = 'https://ycharts.com/companies/TSLA/revenues'

# Configurar encabezados para evitar el bloqueo
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'
}

# Esperar unos segundos antes de la solicitud
time.sleep(4)

# Hacer la solicitud con encabezados
response = requests.get(resource_url, headers=headers)


# Si la petición se ha ejecutado correctamente (código 200), entonces el contenido HTML de la página se ha podido descargar
if response.status_code == 200:
    # Transformamos el HTML plano en un HTML real (estructurado y anidado, con forma de árbol)
    soup = BeautifulSoup(response.text, 'html.parser') # es más actual que usar solo 'html'

soup

tabla = soup.find_all(class_='panel panel-data')
tabla

[<div class="panel panel-data">
 <div class="panel-header">
 <h3 class="panel-title">Revenue (Quarterly) Chart</h3>
 </div>
 <div class="panel-content">
 <div class="chart-cover">
 <div class="chart-interactive">
 <div class="chart-item">
 <div class="chart-options">
 <div class="chart-legend legend-off-chart">
 <div class="chart-legend-item">
 <span class="chart-legend-item-title"></span>
 </div>
 </div>
 </div>
 <div class="chart-wrap">
 <div class="chart-canvas"></div>
 </div>
 </div>
 </div>
 </div>
 </div>
 </div>,
 <div class="panel panel-data">
 <div class="panel-header">
 <h3 class="panel-title">Historical Revenue (Quarterly) Data</h3>
 </div>
 <div class="panel-content">
 <div class="chart-options">
 <div class="panel-ad-cta">
                         View and export this data back to 2008.
                         <a href="/start_trial/histData">Upgrade now.</a>
 </div>
 </div>
 <div class="row">
 <div class="col-6">
 <table class="table">
 <thead>
 <tr>
 <th>Date</th>
 <th c

In [8]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

url = "https://ycharts.com/companies/TSLA/revenues"

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open("revenues.html", "wb") as dataset:
      dataset.write(response.content) 
    print("HTML content downloaded successfully.")
else:
    print(f"Request failed with status code: {response.status_code}")

#Asi lo guardamos.
with open("revenues.html", "r", encoding ="utf-8") as file: # 'utf-8' is common but may need adjustment
  html_content = file.read()
soup = BeautifulSoup(html_content, "html.parser")

#Asi lo levantamos
tables = soup.find_all("table")
tables

# Extraer las dos primeras tablas 
table0 = tables[0]  
table1 = tables[1]

# Extrae los encabezados (headers) de la tabla
headers = []
for th in table0.find_all("th"):
    headers.append(th.text.strip())  # 'strip()' limpia espacios en blanco

# Extrae las filas de datos
data = []
for row in table0.find_all("tr")[1:]:  # Ignora el primer tr si es el header
    cells = row.find_all("td")
    row_data = [cell.text.strip() for cell in cells]
    data.append(row_data)
for row in table1.find_all("tr")[1:]:  # Ignora el primer tr si es el header
    cells = row.find_all("td")
    row_data = [cell.text.strip() for cell in cells]
    data.append(row_data)


# Crea un DataFrame a partir de los datos y encabezados
df = pd.DataFrame(data, columns=headers)

# Muestra el DataFrame resultante
print(df)
df.dtypes

HTML content downloaded successfully.
                  Date    Value
0   September 30, 2024   25.18B
1        June 30, 2024   25.50B
2       March 31, 2024   21.30B
3    December 31, 2023   25.17B
4   September 30, 2023   23.35B
5        June 30, 2023   24.93B
6       March 31, 2023   23.33B
7    December 31, 2022   24.32B
8   September 30, 2022   21.45B
9        June 30, 2022   16.93B
10      March 31, 2022   18.76B
11   December 31, 2021   17.72B
12  September 30, 2021   13.76B
13       June 30, 2021   11.96B
14      March 31, 2021   10.39B
15   December 31, 2020   10.74B
16  September 30, 2020   8.771B
17       June 30, 2020   6.036B
18      March 31, 2020   5.985B
19   December 31, 2019   7.384B
20  September 30, 2019   6.303B
21       June 30, 2019    6.35B
22      March 31, 2019   4.541B
23   December 31, 2018   7.226B
24  September 30, 2018   6.824B
25       June 30, 2018   4.002B
26      March 31, 2018   3.409B
27   December 31, 2017   3.288B
28  September 30, 2017   2.985B
29

Date     object
Value    object
dtype: object

In [16]:
import re

# Definimos una función para convertir a numérico
def convert_to_numeric(value):
    # Si el valor contiene 'B', multiplicamos por mil millones (billones)
    if 'B' in value:
        return float(value.replace('B', '').replace(',', '').strip()) * 1e9
    # Si el valor contiene 'M', multiplicamos por un millón
    elif 'M' in value:
        return float(value.replace('M', '').replace(',', '').strip()) * 1e6
    else:
        # En caso de que ya sea un valor numérico sin letras
        return float(value.replace(',', '').strip())

# Aplicamos la función a la columna 'value' (ajusta el nombre si es diferente)
#df['Value'] = df['Value'].apply(convert_to_numeric)

# Redondea la columna 'value' a tres decimales
df['Value'] = df['Value'].round(3)

# Verificamos el resultado
print(df)
print(df.dtypes)

                  Date         Value
0   September 30, 2024  2.518000e+10
1        June 30, 2024  2.550000e+10
2       March 31, 2024  2.130000e+10
3    December 31, 2023  2.517000e+10
4   September 30, 2023  2.335000e+10
5        June 30, 2023  2.493000e+10
6       March 31, 2023  2.333000e+10
7    December 31, 2022  2.432000e+10
8   September 30, 2022  2.145000e+10
9        June 30, 2022  1.693000e+10
10      March 31, 2022  1.876000e+10
11   December 31, 2021  1.772000e+10
12  September 30, 2021  1.376000e+10
13       June 30, 2021  1.196000e+10
14      March 31, 2021  1.039000e+10
15   December 31, 2020  1.074000e+10
16  September 30, 2020  8.771000e+09
17       June 30, 2020  6.036000e+09
18      March 31, 2020  5.985000e+09
19   December 31, 2019  7.384000e+09
20  September 30, 2019  6.303000e+09
21       June 30, 2019  6.350000e+09
22      March 31, 2019  4.541000e+09
23   December 31, 2018  7.226000e+09
24  September 30, 2018  6.824000e+09
25       June 30, 2018  4.002000e+09
2